In [1]:
import pandas as pd
import datetime as dt

In [15]:
data = pd.read_csv('./data/train_data.csv')
data.iloc[-1,0] = '2019/11/24'
data.iloc[-1,:] = data.iloc[-1,:].fillna(0)

In [16]:
data.tail()

,Date,VGSH,JEF,IVZ,KTCC,FBZ,PLOW,JBK,IAC,IVR,...,ASFI,JTA,MNR,TFSL,GTN,TYG,VIRC,BIS,WOOD,MASI
415,2019/10/27,60.417824,18.853819,16.638926,5.97,15.280874,48.024773,29.00,229.559998,13.800565,...,6.95,11.057277,14.815565,19.043219,16.830000,17.788372,4.01,16.251677,64.136101,147.509995
416,2019/11/3,60.288853,19.238792,17.527559,6.23,14.581332,51.991562,30.00,219.949997,14.149504,...,10.10,11.200258,14.385562,19.189333,19.980000,17.243061,4.09,16.082491,65.034508,142.949997
417,2019/11/10,60.397987,18.967628,17.163631,5.96,14.726692,52.336071,28.50,221.820007,13.913970,...,10.25,11.209790,14.543064,19.189333,19.049999,17.111437,3.93,15.754073,65.343956,149.770004
418,2019/11/17,60.388062,20.636860,17.045601,5.52,14.763033,52.089993,28.74,221.639999,14.044824,...,10.43,11.114470,14.463972,19.267258,19.850000,16.888838,3.68,14.709110,64.635216,153.350006
419,2019/11/24,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


In [17]:
data.iloc[:, 0] = pd.to_datetime(data.iloc[:, 0])

In [18]:
data_wide = data.copy()
data_wide.index = data_wide.iloc[:,0]
data_wide = data_wide.drop('Date', axis=1)

In [19]:
data_columns = data_wide.columns

from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
labelEncoder = labelEncoder.fit(data_columns)
data_wide.columns = labelEncoder.transform(data_columns)
data_wide.head()

,3106,1623,1601,1707,1071,2362,1608,1466,1600,3148,...,194,1650,1930,2890,1336,3003,3113,351,3209,1830
Date,,,,,,,,,,,,,,,,,,,,,
2011-11-13,55.942184,17.649006,13.770864,4.77,13.818835,10.198389,12.440549,38.484608,4.836254,8.575465,...,3.721562,4.747546,5.567790,7.262252,1.80,20.079035,1.504111,826.767029,30.918266,18.430309
2011-11-20,55.978844,16.270664,12.719761,4.94,12.744166,10.035272,12.065370,36.252533,4.658076,7.657301,...,3.590744,4.549112,5.473421,7.125532,1.56,19.711763,1.474619,769.836304,29.410889,17.534525
2011-11-27,56.021038,18.813454,14.402946,4.82,13.918125,10.510439,12.285713,39.479744,4.966706,8.611118,...,3.608788,4.851724,5.372760,7.616116,1.70,20.698137,1.572927,672.212830,31.723339,19.631044
2011-12-04,56.002697,18.425301,14.758045,4.73,13.994051,10.652282,12.148743,38.856613,4.680348,8.406092,...,3.541123,4.931097,5.498586,7.350718,1.74,20.010817,1.779373,694.906921,31.577734,19.383274
2011-12-11,56.030224,17.676432,13.664331,4.55,13.445042,10.449807,12.053457,38.624111,4.772620,7.719699,...,3.577211,4.734335,5.662159,6.956642,1.74,20.336115,1.297665,702.928040,30.412945,18.201599


In [21]:
lag1 = data_wide.shift(1)
lag2 = data_wide.shift(2)
lag3 = data_wide.shift(3)
lag4 = data_wide.shift(4)

In [22]:
ave4 = data_wide.shift(1).rolling(window=4).mean()
ave8 = data_wide.shift(1).rolling(window=8).mean()

#### あいであ
- industryごとにグルーピングして集計したものに対してラグ特徴量の作成する

In [23]:
# data_long = data_wide.stack()
data_long = data_wide.stack().reset_index()
data_long.columns = ['date', 'symbol', 'stockPrice']
data_long.head()

,date,symbol,stockPrice
0,2011-11-13,3106,55.942184
1,2011-11-13,1623,17.649006
2,2011-11-13,1601,13.770864
3,2011-11-13,1707,4.770000
4,2011-11-13,1071,13.818835


In [24]:
def wide_to_long(df_wide):
    df_long = df_wide.stack().reset_index()
    df_long.columns = ['date', 'symbol', 'stockPrice']
    return df_long

In [25]:
feature_df_list = list(map(wide_to_long, [lag1, lag2, lag3, lag4, ave4, ave8]))

In [26]:
data_long_merged = data_long.copy()
for feature_df in feature_df_list:
    data_long_merged = pd.merge(data_long_merged, feature_df, how='left', on=['date', 'symbol'])

/tmp/ipykernel_1139/943731102.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'stockPrice_x'} in the result is deprecated and will raise a MergeError in a future version.
  data_long_merged = pd.merge(data_long_merged, feature_df, how='left', on=['date', 'symbol'])


In [27]:
data_long_merged[data_long_merged['symbol'] == 3106].head(10)
data_long_merged.columns = ['date', 'symbol', 'stockPrice', 'stockPriceLag1', 'stockPriceLag2', 'stockPriceLag3', 'stockPriceLag4',
                           'stockPriceAve4', 'stockPriceAve8']

In [28]:
data_long_merged_drop = data_long_merged.dropna(how='any')

In [30]:
data_long_merged_drop

,date,symbol,stockPrice,stockPriceLag1,stockPriceLag2,stockPriceLag3,stockPriceLag4,stockPriceAve4,stockPriceAve8
26224,2012-01-08,3106,56.000862,55.982456,56.046795,55.991684,56.030224,56.012790,55.999490
26225,2012-01-08,1623,20.411310,19.160168,18.237854,18.887484,17.676432,18.490484,18.140045
26226,2012-01-08,1601,15.354620,14.750947,14.268006,14.360333,13.664331,14.260905,14.086904
26227,2012-01-08,1707,5.490000,5.210000,4.690000,4.580000,4.550000,4.757500,4.786250
26228,2012-01-08,1071,14.594657,14.038782,13.660316,13.772671,13.445042,13.729203,13.673998
...,...,...,...,...,...,...,...,...,...
1376755,2019-11-24,3003,0.000000,16.888838,17.111437,17.243061,17.788372,17.257927,17.685332
1376756,2019-11-24,3113,0.000000,3.680000,3.930000,4.090000,4.010000,3.927500,3.928750
1376757,2019-11-24,351,0.000000,14.709110,15.754073,16.082491,16.251677,15.699338,17.328984
1376758,2019-11-24,3209,0.000000,64.635216,65.343956,65.034508,64.136101,64.787445,62.587602


In [35]:
# splitDate = max(data.iloc[:, 0]) - dt.timedelta(days=28)
# train = data_long_merged_drop[data_long_merged_drop['date'] <= splitDate]
# test = data_long_merged_drop[data_long_merged_drop['date'] > splitDate]

splitDate = dt.datetime.strptime('2019-11-24', '%Y-%m-%d')
train = data_long_merged_drop[data_long_merged_drop['date'] < splitDate]
test = data_long_merged_drop[data_long_merged_drop['date'] >= splitDate]

In [38]:
train_x = train.drop(['date', 'stockPrice'], axis=1)
train_y = train['stockPrice']
test_x = test.drop(['date', 'stockPrice'], axis=1)
test_y = test['stockPrice']

In [39]:
import xgboost as xgb

In [64]:
train_data = xgb.DMatrix(train_x, label=train_y)
eval_data = xgb.DMatrix(test_x, label=test_y)

xgb_params = {
    "objective": "reg:squarederror",
    'eval_metric': "mae"
    }
evals = [(train_data, 'train'), (eval_data, 'eval')]

# gbm = xgb.train(
#     xgb_params,
#     train_data,
#     num_boost_round=100,
#     early_stopping_rounds=10,
#     evals=evals,
#     )

gbm = xgb.train(
    xgb_params,
    train_data,
    num_boost_round=50
    )

In [65]:
predict_y = gbm.predict(eval_data)
predict_y.shape

(3278,)

In [66]:
result = pd.DataFrame([labelEncoder.inverse_transform(test_x['symbol']), predict_y]).T
result.columns = ['id', 'y']
result.to_csv('./data/predict/nb_003_xgboost.csv',index=False)